In [1]:
import pandas as pd
import numpy as np
import scipy
import os

RQ1_DATA_FOLDER_PATH = "../experiment_data/RQ1_data/"

subfolders = [f for f in os.listdir(RQ1_DATA_FOLDER_PATH)
              if os.path.isdir(os.path.join(RQ1_DATA_FOLDER_PATH, f))]

rq_1_df = pd.DataFrame()

for folder in subfolders:
    xlsx_files = [f for f in os.listdir(
        RQ1_DATA_FOLDER_PATH+folder) if f.endswith(".xlsx")]
    for xlsx_file in xlsx_files:
        xlsx_file_path = RQ1_DATA_FOLDER_PATH + f"{folder}/{xlsx_file}"
        temp_df = pd.read_excel(xlsx_file_path)
        temp_df["participant"] = folder
        rq_1_df = pd.concat([rq_1_df, temp_df], ignore_index=True)



### Data Cleaning

In [2]:
column_name_mapper = {
    "Tempo di completamento": "tempo_completamento", 
    "Tool usato?": "tool_used"
}

rq_1_df = rq_1_df.rename(columns=column_name_mapper)    

In [3]:
import pandas as pd
import re

def normalize_time(x):
    if pd.isna(x):
        return None
    
    # Converti in stringa e ripulisci
    s = str(x).strip().replace('"', '').replace("'", "")
    
    # Separa con ":" invece di "."
    s = s.replace(".", ":")
    
    # Caso: HH:MM:SS
    if re.match(r"^\d{1,2}:\d{1,2}:\d{1,2}$", s):
        h, m, sec = s.split(":")
        if h == "00":
            total_seconds = int(m) * 60 + int(sec)    
        else:
            total_seconds = int(h) * 60 + int(m)
    
    # Caso: M:SS,ms → ignoro millisecondi
    elif re.match(r"^\d{1,2}:\d{1,2},\d+$", s):
        m, rest = s.split(":")
        sec = rest.split(",")[0]
        total_seconds = int(m) * 60 + int(sec)
    
    # Caso: M:SS
    elif re.match(r"^\d{1,2}:\d{1,2}$", s):
        m, sec = s.split(":")
        total_seconds = int(m) * 60 + int(sec)
    
    # Caso: solo numero (minuti)
    elif s.isdigit():
        total_seconds = int(s) * 60
    
    else:
        return "None"
    
    # Conversione in formato MM:SS
    minutes, seconds = divmod(total_seconds, 60)
    return f"{minutes:02d}:{seconds:02d}"

In [4]:
def count_seconds(x):
    if pd.isna(x):
        return None
    time: str = x
    m, sec = time.split(":")
    m = int(m)
    sec = int(sec)
    return m*60 + sec


In [5]:
rq_1_df.columns

Index(['Script', 'Descrizione', 'tempo_completamento', 'tool_used',
       'participant'],
      dtype='object')

In [6]:
rq_1_df["tempo_completamento_normalized"] = rq_1_df["tempo_completamento"].apply(normalize_time)

In [7]:
righe_na = rq_1_df[rq_1_df["tempo_completamento_normalized"].isna()]

In [8]:
rq_1_df = rq_1_df.dropna(axis=0)

In [9]:
rq_1_df["tempo_completamento_normalized_seconds"] = rq_1_df["tempo_completamento_normalized"].apply(count_seconds)

### Save Dataframe as csv

In [10]:
#rq_1_df.to_csv(f"{RQ1_DATA_FOLDER_PATH}/all_data_cleaned.csv")

### Check Script 1 

In [11]:
tool_used_df_script1 = rq_1_df[rq_1_df["tool_used"]==True]
tool_used_df_script1.describe()

,tempo_completamento_normalized_seconds
count,30.000000
mean,234.366667
std,121.306248
min,36.000000
25%,142.500000
50%,217.500000
75%,298.750000
max,510.000000


In [12]:
tool_not_used_df_script1 = rq_1_df[rq_1_df["tool_used"]==False]
tool_not_used_df_script1.describe()

,tempo_completamento_normalized_seconds
count,35.000000
mean,326.114286
std,204.285379
min,45.000000
25%,216.000000
50%,299.000000
75%,386.000000
max,1188.000000


In [13]:
from scipy.stats import shapiro

shapiro(tool_used_df_script1["tempo_completamento_normalized_seconds"])

ShapiroResult(statistic=np.float64(0.9662832605535963), pvalue=np.float64(0.44308691512798504))

In [14]:
shapiro(tool_not_used_df_script1["tempo_completamento_normalized_seconds"])

ShapiroResult(statistic=np.float64(0.8161083509179551), pvalue=np.float64(4.290212302083819e-05))

In [15]:
from scipy.stats import mannwhitneyu

stat, p_value = mannwhitneyu(
    tool_used_df_script1["tempo_completamento_normalized_seconds"],
    tool_not_used_df_script1["tempo_completamento_normalized_seconds"],
    #alternative="two-sided"  # due code, per testare differenze in entrambe le direzioni
)

print("Statistic:", stat)
print("p-value:", p_value)

if p_value < 0.05:
    print("Possiamo rigettare ipotesi nulla")
else:
    print("NON Possiamo rigettare ipotesi nulla")

Statistic: 358.0
p-value: 0.028434246379173904
Possiamo rigettare ipotesi nulla
